In [1]:
#Python libraries for web scraping
from bs4 import BeautifulSoup
import requests
import re

#Python libraries for NLP tasks
import pandas as pd
import numpy as np

In [2]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
url = 'https://www.boxofficemojo.com/year/{}/?area=VN&grossesOption=calendarGrosses'

In [3]:
df_calendar_gross = pd.DataFrame(columns=['Year','Rank','MovieID','Gross'])
df_movie = pd.DataFrame(columns=['MovieID','Movie','Title_Summary','Country_of_origin','Language','Distributor','Release_Date','MPAA','Running_Time','Genres','Rating','Votes','Theaters','Budget','Total_Gross','Domestic','Worldwide','Image','IMDb_url'])
df_weekend_gross = pd.DataFrame(columns=['MovieID','Date','Calendar_Week','Rank','Compare_LW','Theaters','Change','To_Date','Weekend'])

In [4]:
df_movie =pd.read_csv('movie.csv')
df_calendar_gross =pd.read_csv('calendar_gross.csv')
df_weekend_gross =pd.read_csv('weekend_gross.csv')


In [12]:
year_from = 2011
year_to = 2008

suffix={'K':1000,'M':1000000}

for year in range(year_from, year_to,-1):
    url_yr = url.format(year)
   
    main_page = requests.get(url_yr, headers=headers)
    if main_page.status_code ==200:
        main_page_src = BeautifulSoup(main_page.content,'html.parser')
        main_page_src 
        main_page_tbl = main_page_src.find('div',{'class':'a-section imdb-scroll-table-inner'}).find('table')
        main_page_rows = main_page_tbl.find_all('tr')
        for i in range(1,len(main_page_rows)):
            
            main_page_cols = main_page_rows[i].find_all('td')
            
            movie_href = main_page_cols[1].find('a')['href']
            movie_url = "https://www.boxofficemojo.com"+ movie_href
            movie_page = requests.get(movie_url, headers=headers)
            movie_src = BeautifulSoup(movie_page.content,'html.parser')
        
            #get the movie detail page
            movie_img = movie_src.find('div',{'class':'a-section a-spacing-none mojo-posters'}).find('img')['src']
            movie_desc = movie_src.find('div',{'class':'a-fixed-left-grid-col a-col-right'}).find('p')
            movieid = movie_href.split("/")[2].strip()
            
            
            movie_desc = "" if movie_desc is None else movie_desc.text
                    
            attrs = movie_src.find('div',{'class':'a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile'}).find_all('div',{'class':'a-section a-spacing-none'})
            
            movie_detail = {}
            
            for attr in attrs:
                key = attr.find_all('span')[0]
                val = attr.find_all('span')[1]
                movie_detail[key.text] = val.text.replace('\n','')
          
        
            if key.text.strip() == "IMDbPro":
                imdb_url = val.find('a')['href'].replace('pro.','')
            
            movie_gross = {}
            attrs = movie_src.find('div',{'class':'a-section a-spacing-none mojo-performance-summary-table'}).find_all('div',{'class':'a-section a-spacing-none'})
       
            movie_gross['Total_Gross'] = attrs[0].find_all('span')[1].text.replace('\n','').strip()
            movie_gross['Domestic'] = attrs[1].find_all('span')[2].text.replace('\n','').strip()
            movie_gross['Worldwide'] = attrs[2].find_all('span')[1].text.replace('\n','').strip()
         
                        
            #find the movie rating        
            rating_page = requests.get(imdb_url, headers=headers)
            rating_src = BeautifulSoup(rating_page.content,'html.parser')
            rating = rating_src.find('div',{'data-testid':'hero-rating-bar__aggregate-rating__score'})
            
            rating_score, rating_votes = 0,0
            lst_country_of_origin, lst_languages = 'Not Supplied','Not Supplied'
            
            if rating:
                rating_score = float(rating.find('span').text)
                
                txt_tmp = rating_src.find('div',{'class':'sc-bde20123-3 gPVQxL'}).text.strip()
                if txt_tmp[-1] in suffix:
                    rating_votes = float(txt_tmp[:-1])*float(suffix[txt_tmp[-1]])
                else:
                    rating_votes = float(txt_tmp)
                
            #title detail information
            rating = rating_src.find('li',{'data-testid':'title-details-origin'})
            
            if rating:
                    
                country_of_origin = rating.find_all('li',{'class':'ipc-inline-list__item'})
                lst_country_of_origin = ''.join(c.text +'-' for c in country_of_origin)
                lst_country_of_origin= lst_country_of_origin[:-1]
                         
            rating = rating_src.find('li',{'data-testid':'title-details-languages'})
            if rating:
                languages = rating.find_all('li',{'class':'ipc-inline-list__item'})
                lst_languages = ''.join(l.text + '-' for l in languages)
                lst_languages = lst_languages[:-1]
            
                   
            #update dataframe
            dict_calendar_gross = {'Year':int(year),
                                   'Rank':int(main_page_cols[0].text),
                                   'MovieID':movieid,
                                   'Gross':int(main_page_cols[5].text.replace('$','').replace(',',''))
                                   }
            
            df_calendar_gross=df_calendar_gross.append(dict_calendar_gross,ignore_index=True)
            
         
            #check if the movie is not in the movie list then add it 
            if df_movie.loc[df_movie['MovieID']==movieid].empty:
              
                dict_movie = {
                              'MovieID':movieid,
                              'Movie':main_page_cols[1].text, 
                              'Title_Summary':movie_desc,
                              'Country_of_origin':lst_country_of_origin,
                              'Language': lst_languages,
                              'Distributor':movie_detail["Distributor"].replace('n/a','Not Supplied'),
                              'Release_Date':"" if "Release Date" not in movie_detail else movie_detail["Release Date"], 
                              'MPAA': "Not Supplied" if "MPAA" not in movie_detail else movie_detail["MPAA"], 
                              'Running_Time':"" if "Running Time" not in movie_detail else movie_detail["Running Time"], 
                              'Genres':"" if "Genres" not in movie_detail else re.sub('  +','-',movie_detail["Genres"]),
                              'Rating':rating_score,
                              'Votes': rating_votes,
                              'Theaters': 0 if main_page_cols[6].text=='-' else int(main_page_cols[6].text.replace(',','')),
                              'Budget': 0 if "Budget" not in movie_detail else movie_detail["Budget"].replace('$','').replace(',',''),
                              'Total_Gross':movie_gross["Total_Gross"].replace('$','').replace(',',''),
                              'Domestic':0 if movie_gross["Domestic"]=='–' else movie_gross["Domestic"].replace('$','').replace(',',''),
                              'Worldwide':movie_gross["Worldwide"].replace('$','').replace(',',''),
                              'Image':movie_img,
                              'IMDb_url':imdb_url
                             }
              
        
                df_movie=df_movie.append(dict_movie, ignore_index=True)
            
            if df_weekend_gross.loc[df_weekend_gross['MovieID']==movieid].empty:
            
                #get the weekend gross movement
                weekend_rows = movie_src.find('div',{'class':'a-section imdb-scroll-table-inner'}).find_all('tr')
                for i in range(1, len(weekend_rows)):
                    weekend_cols = weekend_rows[i].find_all('td')  
                    compare_lw = 0 if weekend_cols[3].text == '-' else int(weekend_cols[3].text.replace('+','').replace('.','').replace(',','').replace('%',''))/100
                                     
                    dict_weekend = {
                                    "MovieID":movieid,
                                    "Date":weekend_cols[0].text,
                                    "Calendar_Week":weekend_cols[0].find("a")["href"].split("/")[2],
                                    "Rank":0 if weekend_cols[1].text=='-' else int(weekend_cols[1].text),
                                    "Weekend_Gross":int(weekend_cols[2].text.replace('$','').replace(',','')),
                                    "Compare_LW":compare_lw,
                                    "Theaters":0 if weekend_cols[4].text=='-' else int(weekend_cols[4].text.replace('$','').replace(',','')),
                                    "Change": 0 if weekend_cols[5].text=='-' else int(weekend_cols[5].text.replace(',','')),
                                    "To_Date":0 if weekend_cols[7].text=='-' else int(weekend_cols[7].text.replace('$','').replace(',','')),
                                    "Weekend":0 if weekend_cols[8].text=='-' else int(weekend_cols[8].text.replace('$','').replace(',',''))
                                   }
                    df_weekend_gross = df_weekend_gross.append(dict_weekend, ignore_index=True)

In [16]:
df_movie

,MovieID,Movie,Title_Summary,Country_of_origin,Language,Distributor,Release_Date,MPAA,Running_Time,Genres,Rating,Votes,Theaters,Budget,Total_Gross,Domestic,Worldwide,Image,IMDb_url
0,rl1985118209,Mai,"Restlessly haunted by the past, Mai is greeted...",Vietnam,Vietnamese,CJ EntertainmentSee full company information,"Feb 10, 2024",Not Supplied,2 hr 11 min,Drama-Romance,6.8,424.0,11129,0,22184037,0,22236306,https://m.media-amazon.com/images/M/MV5BMjAwMG...,https://imdb.com/title/tt31174028?ref_=mojo_rl...
1,rl945586177,Exhuma,The process of excavating an ominous grave unl...,South Korea,Korean,Galaxy Studio,"Mar 13, 2024",Not Supplied,2 hr 14 min,Horror-Mystery-Thriller,7.4,2200.0,11123,0,6611268,1321130,86869015,https://m.media-amazon.com/images/M/MV5BMzczMm...,https://imdb.com/title/tt27802490?ref_=mojo_rl...
2,rl844660737,Kung Fu Panda 4,After Po is tapped to become the Spiritual Lea...,United States-China,Mandarin-Chinese-English,Galaxy Studio,"Mar 8, 2024",PG,1 hr 34 min,Action-Adventure-Animation-Comedy-Family-Fantasy,6.5,17000.0,9718,0,4684354,153781205,348066205,https://m.media-amazon.com/images/M/MV5BZDY0Yz...,https://imdb.com/title/tt21692408?ref_=mojo_rl...
3,rl2337374209,Gap Lai Chi Bau,NaN,Not Supplied,Not Supplied,Not Supplied,"Feb 2, 2024",Not Supplied,1 hr 54 min,Comedy-Romance,0.0,0.0,2628,0,3749297,0,3749297,https://m.media-amazon.com/images/M/MV5BMGNlMj...,https://imdb.com/title/tt31180892?ref_=mojo_rl...
4,rl2974318593,Quy Cau,The return of Nam and his girlfriend Xuan brin...,Vietnam,Vietnamese,Galaxy Studio,"Dec 22, 2023",Not Supplied,1 hr 39 min,Horror,6.7,70.0,4382,0,4421932,0,4421932,https://m.media-amazon.com/images/M/MV5BNzJlNW...,https://imdb.com/title/tt30611573?ref_=mojo_rl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,rl1136559873,Grown Ups,After their high school basketball coach passe...,United States,English-Spanish,Sony Pictures ReleasingSee full company inform...,"Sep 3, 2010",PG-13,1 hr 42 min,Comedy,6.0,275000.0,1,80000000,212451,162001186,271430189,https://m.media-amazon.com/images/M/MV5BMjA0OD...,https://imdb.com/title/tt1375670?ref_=mojo_rl_...
1239,rl2944238593,Terminator Salvation,"In 2018, a mysterious new weapon in the war ag...",United States-Germany,English-Italian,Sony Pictures ReleasingSee full company inform...,"Jun 12, 2009",PG-13,1 hr 55 min,Action-Adventure-Sci-Fi,6.5,377000.0,18,200000000,276201,125322469,371353001,https://m.media-amazon.com/images/M/MV5BODBlOT...,https://imdb.com/title/tt0438488?ref_=mojo_rl_...
1240,rl2217182721,Angels & Demons,Harvard symbologist Robert Langdon works with ...,United States-Italy,English-Italian-Latin-French-Swiss German-Germ...,Sony Pictures ReleasingSee full company inform...,"Jun 26, 2009",PG-13,2 hr 18 min,Action-Mystery-Thriller,6.7,309000.0,18,150000000,292350,133375846,485930816,https://m.media-amazon.com/images/M/MV5BMjEzNz...,https://imdb.com/title/tt0808151?ref_=mojo_rl_...
1241,rl2222556161,The Taking of Pelham 123,"Armed men hijack a New York City subway train,...",United States-United Kingdom,English-Ukrainian,Sony Pictures ReleasingSee full company inform...,"Aug 7, 2009",R,1 hr 46 min,Action-Crime-Thriller,6.4,208000.0,18,100000000,116036,65452312,150166126,https://m.media-amazon.com/images/M/MV5BMTU3Nz...,https://imdb.com/title/tt1111422?ref_=mojo_rl_...


In [13]:
df_calendar_gross

,Year,Rank,MovieID,Gross
0,2024,1,rl1985118209,22184037
1,2024,2,rl945586177,5797480
2,2024,3,rl844660737,4684354
3,2024,4,rl2337374209,3749297
4,2024,5,rl2974318593,2917999
...,...,...,...,...
1317,2010,9,rl1136559873,24
1318,2009,1,rl2944238593,276201
1319,2009,2,rl2217182721,161933
1320,2009,3,rl2222556161,116036


In [15]:
df_weekend_gross

,MovieID,Date,Calendar_Week,Rank,Compare_LW,Theaters,Change,To_Date,Weekend,Weekend_Gross
0,rl1985118209,Feb 9-11,2024W06,1,0.00,6784,0,2055973,1,2055973.0
1,rl1985118209,Feb 16-18,2024W07,1,14.22,13,-6771,13270246,2,4979775.0
2,rl1985118209,Feb 23-25,2024W08,1,-5.46,11129,11116,18956539,3,2261023.0
3,rl1985118209,Mar 1-3,2024W09,1,-0.62,7190,-3939,21007739,4,859056.0
4,rl1985118209,Mar 8-10,2024W10,2,-0.44,3939,-3251,21950339,5,481120.0
...,...,...,...,...,...,...,...,...,...,...
4601,rl2217182721,Jul 17-19,2009W29,2,-7.93,3,-2,292350,4,124.0
4602,rl2222556161,Aug 7-9,2009W32,1,0.00,18,0,36044,1,36044.0
4603,rl2222556161,Aug 14-16,2009W33,1,-4.92,18,0,84058,2,18327.0
4604,rl2222556161,Sep 4-6,2009W36,1,0.00,1,0,116036,5,375.0


In [14]:
df_movie.to_csv("movie.csv",index=False)
df_weekend_gross.to_csv("weekend_gross.csv",index=False)
df_calendar_gross.to_csv('calendar_gross.csv',index=False)